In [8]:
import sqlite3
import json
import os
import re
import pandas as pd


In [4]:
# Top Chess Players in the world as of December 16, 2024
# Ordering within the dictionary does not matter
# Temporary way of accessing everything
top_players =[
    {'name': 'Magnus_Carlsen', 'fide_number': '1503014', 'chess_com_username': 'magnuscarlsen'},
    {'name': 'Fabiano_Caruana', 'fide_number': '2020009', 'chess_com_username': 'fabianocaruana'},
    {'name': 'Hikaru_Nakamura', 'fide_number': '2016192', 'chess_com_username': 'hikaru'},
    {'name': 'Arjun_Erigaisi', 'fide_number': '35009192', 'chess_com_username': 'ghandeevam2003'},
    {'name': 'Gukesh_Dommaraju', 'fide_number': '46616543', 'chess_com_username': 'gukeshdommaraju'},
    {'name': 'Nodirbek_Abdusattorov', 'fide_number': '14204118', 'chess_com_username': 'chesswarrior7197'},
    {'name': 'Alireza_Firouzja', 'fide_number': '12573981', 'chess_com_username': 'firouzja2003'},
    {'name': 'Ian_Nepomniachtchi', 'fide_number': '4168119', 'chess_com_username': 'lachesisq'},
    {'name': 'Yi_Wei', 'fide_number': '8603405', 'chess_com_username': 'wei-yi'},
    {'name': 'Viswanathan_Anand', 'fide_number': '5000017', 'chess_com_username': 'thevish'}
]

## Database Origin ##

This pathway will be used in all functions as the dedicated location of the database we work on

In [12]:
db_path = "../data/chess.db"

# Fide Database Clean Up # 

In this section we will write a function which receives a json file from the Fide folder and moves the data into a SQL database with 2 tables.

The first table will hold non-timeseries data from the FIDE site (IE rankings, country, etc) while the other will focus on their historical elo progression.

In [22]:
def process_player_json(json_file, database_path= db_path):
    """
    Processes a single player's JSON file and updates the database.
    Creates/updates entry in shared player_info table and maintains a unified player_history table.

    Args:
        json_file (str): Path to the JSON file for the player.
        database_path (str): Path to the SQLite database file.

    Returns:
        bool: True if successful, False if an error occurred.
    """
    try:
        # Read the JSON file
        with open(json_file, "r") as file:
            data = json.load(file)

        # Extract history and player info
        history_data = data.pop("history")
        player_info = data

        # Connect to the database
        with sqlite3.connect(database_path) as conn:
            cursor = conn.cursor()

            # Enable foreign key support
            cursor.execute("PRAGMA foreign_keys = ON")

            # Create or update the shared player_info table
            cursor.execute("""
                CREATE TABLE IF NOT EXISTS player_info (
                    player_id INTEGER PRIMARY KEY AUTOINCREMENT,
                    name TEXT UNIQUE,
                    federation TEXT,
                    birth_year INTEGER,
                    sex TEXT,
                    title TEXT,
                    world_rank_all_players INTEGER,
                    world_rank_active_players INTEGER,
                    national_rank_all_players INTEGER,
                    national_rank_active_players INTEGER,
                    continental_rank_all_players INTEGER,
                    continental_rank_active_players INTEGER
                )
            """)

            # Update or insert player info
            cursor.execute("""
                INSERT OR REPLACE INTO player_info (
                    name, federation, birth_year, sex, title, standard_elo, rapid_elo, blitz_elo,
                    world_rank_all_players, world_rank_active_players,
                    national_rank_all_players, national_rank_active_players,
                    continental_rank_all_players, continental_rank_active_players
                ) VALUES (
                    :name, :federation, :birth_year, :sex, :title, :standard_elo, :rapid_elo, :blitz_elo,
                    :world_rank_all_players, :world_rank_active_players,
                    :national_rank_all_players, :national_rank_active_players,
                    :continental_rank_all_players, :continental_rank_active_players
                )
            """, player_info)

            # Get the player_id
            cursor.execute("SELECT player_id FROM player_info WHERE name = ?", (player_info['name'],))
            player_id = cursor.fetchone()[0]

            # Create or update the unified player_history table
            cursor.execute("""
                CREATE TABLE IF NOT EXISTS player_history (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    player_id INTEGER,
                    date TEXT,
                    numeric_date INTEGER,
                    standard TEXT,
                    num_standard_games TEXT,
                    rapid TEXT,
                    num_rapid_games TEXT,
                    blitz TEXT,
                    num_blitz_games TEXT,
                    FOREIGN KEY (player_id) REFERENCES player_info(player_id)
                )
            """)

            # Add player_id to each history record
            for record in history_data:
                record['player_id'] = player_id

            # Insert new history data
            cursor.executemany("""
                INSERT OR REPLACE INTO player_history (
                    player_id, date, numeric_date, standard, num_standard_games,
                    rapid, num_rapid_games, blitz, num_blitz_games
                ) VALUES (
                    :player_id, :date, :numeric_date, :standard, :num_standard_games,
                    :rapid, :num_rapid_games, :blitz, :num_blitz_games
                )
            """, history_data)

        print(f"Successfully processed data for '{player_info['name']}'")
        return True

    except json.JSONDecodeError as e:
        print(f"Error reading JSON file {json_file}: {e}")
        return False
    except sqlite3.Error as e:
        print(f"Database error occurred: {e}")
        return False
    except Exception as e:
        print(f"Unexpected error occurred: {e}")
        return False


The loop below iterates through each of the json files and uses the process_player_json() to create a singular FIDE database 

In [23]:
for player in top_players:
    name = player["name"]
    process_player_json(f"/files/ds105a-2024-project-fork_force/data/Fide/{name}_fide.json")

Successfully processed data for 'Carlsen, Magnus '
Successfully processed data for 'Caruana, Fabiano '
Successfully processed data for 'Nakamura, Hikaru '
Successfully processed data for 'Erigaisi Arjun '
Successfully processed data for 'Gukesh D '
Successfully processed data for 'Abdusattorov, Nodirbek '
Successfully processed data for 'Firouzja, Alireza '
Successfully processed data for 'Nepomniachtchi, Ian '
Successfully processed data for 'Wei, Yi '
Successfully processed data for 'Anand, Viswanathan '
